# **MDX 데이터 분석하기**
https://pypi.org/project/mdict-utils/
1. **MDX 사전파일을** 활용하여 객체에 구분하기
1. 각 사전의 첫번째 의미를 통해서 **NER** 사전 만들기
1. 추후에 보완을 해서 wordnet 으로 강화하기

## **1 Sqlite3 데이터 불러오기**
sqlite3 데이터 불러오기

In [1]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('backup/kordict.db')
resp = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
tableNames = [name[0]  for name in resp]
tableNames

['meta', 'mdx']

In [2]:
%%time
# 색인정보 : 51만개의 단어사전 데이터
df = pd.read_sql_query("SELECT * FROM {}".format(tableNames[1]), conn)
itemIndex = df.entry.values.tolist()
itemData  = df.paraphrase.values.tolist()
print(len(itemIndex), itemIndex[:10], df.shape)

516490 ['ᄛ', 'ᄻ', 'ᆐ', 'ㄱ', 'ㄱ', 'ㄱㄴㄷ순', 'ㄱㄴㄷ차례', 'ㄱㄴ순', 'ㄱㄴ차례', 'ㄱ자'] (516490, 2)
CPU times: user 1.29 s, sys: 211 ms, total: 1.5 s
Wall time: 1.5 s


## **2 HTML 태그 제거 및 본문선별**
가장 처음 제시된 **Tag 내용과 개념** 정리
1. **/font** 태그가 제목 이외에, 세부내용에도 포함됩니다.
1. 때문에 이를 무조건 적용하기엔 문제가 있어서 Tag 구조를 명확하게 파악하기
1. 우선 구조에 대한 형태를 몇가지로 확인한 뒤 작업을 진행하기

In [3]:
# 해당 Token 의 Index 찾기
# search_query = "가지"  # "가결" # "서버"  # "가결", "대리인"
def wordData(search_query):
    return [no  for no, _ in enumerate(itemIndex)  if _ == search_query]

In [4]:
# 텍스트 인덱스의 본문내용 찾기
import re
def wordDetails(_xmlIdx):
    _xml = itemData[_xmlIdx].lower()
    regex_html = '<.*?>'                 # html 태그내용만 추출
    regex_text = "<[^>]+>|[^<]+"         # html 태그 내부의 한글 추출
    regex_font = '<font.*?>(.+?)</font>' # html 추출 : font 태그 내용만 추출
    r_full = re.findall(regex_text, _xml)
    r_html = re.findall(regex_html, _xml) # HTML 태그내용
    r_texts =  [_  for _ in r_full  if _ not in r_html]
    
    # 위 결과를 활용하여 의미별 구분 인덱스 계산
    r_cate = re.compile(r'「(.*?)」')
    r_num  = re.compile(r'\d+')
    r_txt  = re.compile(r'[·가-힣]+')
    _cate, _temp = [], []
    
    for no, _ in enumerate(r_texts):
        _ = "".join(r_cate.findall(_))
        
        if _: # 유효값이 추출된 경우
            _temp.append(_) # 유효값 저장객체
            _num = r_num.findall(_)
            
            if _num:  # 숫자값이 추출된 경우
                for _t in _temp[::-1]:
                    _txt = r_txt.findall(_t)
                    if _txt:  _ = _txt[0]
            _cate.append((no, _))        
    return _cate, r_texts

In [5]:
def exportMeans(t_temp, idx_cate):
    # 인덱스 값이 추출된 경우
    if idx_cate:
        _idxno = [_[0]  for _ in idx_cate]
        _idxno.append(len(t_temp))

        # n-gram 함수를 응용 2개쌍 작업용 주소값 호출
        _idxno = [(idx_cate[_][1] ,_idxno[_: _+2]) 
                   for _ in range(len(_idxno)-1)]

        # + 1 : 분류 기준의 제거 (그런데 분류기준에 Text 설명이 포함되기도 함.. ㅜㅜ)
        result = []
        for _ in _idxno:
            _temp = t_temp[_[1][0] : _[1][1]]
            _temp = [_.replace("「","").replace("」","") for _ in _temp]
            if _temp:  # 동일하면 제외
                if (len(_temp) == 1) and (_temp[0] == _[0]): pass
                elif (len(_temp) == 1): result.append((_[0], _temp[0]))  
                else: result.append((_[0], _temp))
        return result
    
    # 별도 인덱스가 없는 경우 1줄로 모두 출력하기
    else:
        return [("명사", ["명사"] + t_temp)]

In [6]:
s_query = "가능"
r_xml   = wordData(s_query)
print("검색결과 :", len(r_xml), r_xml)
r_idx   = wordDetails(r_xml[0])
exportMeans(r_idx[1], r_idx[0])

검색결과 : 1 [1244]


[('명사',
  ['명사',
   '할 수 있거나 될 수 있음.',
   '¶ 통화 가능 지역/이 세상엔 가능이 있을 뿐 해서 안 되는 일은 없사옵니다.≪박종화, 전야≫',
   '가능-하다(可能--) [가ː---]']),
 ('형용사',
  ['형용사',
   '할 수 있거나 될 수 있다.',
   '¶ 가능한 모든 수단과 방법을 동원했지만 그를 설득할 수는 없었다./모든 일에 가능한 한 최대의 노력을 기울여야 한다./컴퓨터 통신의 발달로 전 세계 사람들과 정보 교환이 가능하게 되었다.']),
 ('반', ' 반불능하다.\r\n')]

## **3 객체의 저장 및 점검**
앞의 단일 분류 결과값 저장 및 점검

In [7]:
# 메모리와 시간이 생각보다 빠르게 처리됨
result = []
from tqdm import tqdm
nums = len(itemData)
for no in tqdm(range(nums)):
    r_idx   = wordDetails(no)
    result.append(exportMeans(r_idx[1], r_idx[0]))
    
nerDict = pd.DataFrame()
nerDict.insert(0, 'Text', itemIndex)
nerDict.insert(1, 'Data', result)

100%|██████████| 516490/516490 [00:17<00:00, 29359.02it/s]


In [ ]:
# 사전중 한글 명사 단어 인덱스 전처리
import re
itemData   = nerDict.Data.values.tolist()
noun_check = [no  for no, _ in enumerate(itemData)  if _[0][0] == "명사"]
nerDict = nerDict.loc[noun_check].reset_index(drop=True)
nerDict.shape

In [ ]:
# 게중 온전한 한글로만 인덱스가 구성된 경우
itemIndex = nerDict.Text.values.tolist()
noun_check = [no  for no, _ in enumerate(itemIndex)   if "".join(re.findall(r"[가-힣]",_)) == _]
nerDict = nerDict.loc[noun_check].reset_index(drop=True)
nerDict.shape

In [8]:
import pickle
with open('data/nerDict.pk', 'wb') as handle:
    pickle.dump(nerDict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# nerDict.to_csv('data/nerDict.csv', index=None)
# nerDict.head()    

In [9]:
s_query = "가능수력"
r_xml   = wordData(s_query)
print("검색결과 :", len(r_xml), r_xml)
r_idx   = wordDetails(r_xml[0])
exportMeans(r_idx[1], r_idx[0])

검색결과 : 1 [1249]


[('명사',
  ['명사',
   '가능수력',
   '가능^수력(可能水力)',
   '『건설』',
   '아직 이용하지는 않으나 설비를 하면 이용할 수 있는 물의 힘.\r\n'])]

## **정규표현식 연습하기**
Regex Tutorials

In [11]:
import re
re.findall('.+', "sales.xls, test.xls")

['sales.xls, test.xls']

In [10]:
re.findall('(<br>|<br/>)', "<br> 텍스트 <br/>")

['<br>', '<br/>']

In [12]:
import re
p = re.compile(r"(?P<name>\w+)\s+(?P<phone>(\d+)[-]\d+[-]\d+)")
print(p.sub("\g<phone> \g<name>", "park 010-1234-1234"))

010-1234-1234 park


In [13]:
text = "$23.24  의 가격은 $69.23 까지 상승하였습니다"
p = re.compile(r"\$[0-9.]+")
p.findall(text)

['$23.24', '$69.23']